In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

import numpy as np
from datetime import datetime
from packaging import version
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.1.0


In [3]:
img_input_data = keras.layers.Input(shape=(12, 128, 256), name="input_imgs")
permute1 = keras.layers.Permute((2,3,1))(img_input_data)
efficientnet = keras.applications.EfficientNetB2(
    include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
    pooling=None, classes=1000, classifier_activation='softmax'
)(permute1)
output = conv2d1
model = keras.models.Model(inputs=[img_input_data], outputs=[output])

# Renaming all layers which deals with image
for layer in model.layers:
    layer._name = "vision_" + layer.name
    
#concat = keras.layers.concatenate([input_data, hidden2])


print(model.summary())

AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.applications' has no attribute 'EfficientNetB2'